In [2]:
from SemanticCorruption import *

In [3]:
from nltk.corpus import wordnet as wn
import pattern.en as en
import numpy as np
import itertools

In [8]:
sents = ["It's not too bad but makes a lot of unnatural sounding sentences.",
'A Washington County man may have the countys first human case of West Nile virus, the health department said Friday',
"It was a dusty, dry, hot day, and the flys were buzzing.",
"The article is the most common determiner (DT) in English.",
"We may have a question",
"Is changing an odd number of verbs, adverbs, adjectives and nouns to their antonyms expected to produce a semantically distant sentence?",
"Both gerunds and infinitives can be used as the subject or the complement of a sentence.",
"Shares of Xoma fell 16 percent in early trade, while shares of Genentech, a much larger company with several products on the market, were up 2 percent.",
"Six months ago, the IMF and Argentina struck a bare-minimum $6.8-billion debt rollover deal that expires in August.",
"He plans to have dinner with troops at Kosovo's U.S. military headquarters, Camp Bondsteel.",
"After that, he plans to have dinner at Camp Bondsteel with U.S. troops stationed there.",
"He added that prosecutors will seek the death penalty.",
"Who is this man?",
"Who is that man?",
"This evil thief stole that car!",
"The motorist is angry, so the pedestrian is understandably scared",
"The hero was brave, he defeated the dragon.",
"The villian was cowardly, he was slain by the dragon.",
"War and Peace opens in the Russian city of St. Petersburg in 1805, as Napoleon's conquest of western Europe is just beginning to stir fears in Russia.",
"PCCW's chief operating officer, Mike Butcher, and the Arena brothers, the chief financial officers, will report directly to the police officer.",
"You sing very well.",
"We look forward to your talk.",
"I knew French.",
         
        ]

In [35]:
!git add ./WSD.ipynb
!git commit -m="Evaluated WSD, good, but not good enough"
!git push

[master 63f350b] =Evaluated WSD, good, but not good enough
 1 file changed, 1385 insertions(+)
 create mode 100644 prototypes/Projects/SemanticVsSyntacticEval/WSD.ipynb
To git@github.com:oxinabox/phd.git
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to 'git@github.com:oxinabox/phd.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


In [57]:

for ii,sent in enumerate(sents):
    print(ii)
    print(sent)
    print("\n".join(leveled_semantic_corrupt_noun_synonym_sentences(sent)))
    print("-"*50)

0
It's not too bad but makes a lot of unnatural sounding sentences.
It 's not too bad but makes a lot of unnatural sounding convictions .
--------------------------------------------------
1
A Washington County man may have the countys first human case of West Nile virus, the health department said Friday
a Washington County serviceman may have the countys first human case of West Nile virus , the health department said Friday
a Washington County serviceman may have the counties first human case of West Nile virus , the health department said Friday
a Washington County serviceman may have the counties first human case of West Nile virus , the health section said Friday
a Washington County serviceman may have the counties first human case of West Nile virus , the wellness section said Friday
a Washington County serviceman may have the counties first human slip of West Nile virus , the wellness section said Friday
--------------------------------------------------
2
It was a dusty, dry, 

In [4]:
import sys
sys.path.append("../../../Resources/tools/lib/python/pywsd/")
import pywsd
from pywsd import disambiguate
from pywsd.similarity import max_similarity as maxsim

In [5]:
def make_pywsd_sent(tagged_words):
    return[(word,word,pennPOS2WordnetPOS(tag)) for (word,tag) in tagged_words]

In [8]:
import nltk.wsd


sent = sents[5]
words, tagged_words = tokenize_and_tag(sent)
wsd_sent =make_pywsd_sent(get_tagged_phrases(tagged_words,3))
ans=disambiguate(wsd_sent, context_is_lemmatized=True, nbest=True, normalizescore=True, keepscore=True, algorithm=pywsd.lesk.simple_lesk)
ans

(u'Is', u'Is', None)
(u'changing', u'changing', u'v')
(u'an', u'an', None)
(u'odd', u'odd', u'a')
(u'number', u'number', u'n')
(u'of', u'of', None)
(u'verbs', u'verbs', u'n')
(u',', u',', None)
(u'adverbs', u'adverbs', u'n')
(u',', u',', None)
(u'adjectives', u'adjectives', u'n')
(u'and', u'and', None)
(u'nouns', u'nouns', u'n')
(u'to', u'to', None)
(u'their', u'their', None)
(u'antonyms', u'antonyms', u'n')
(u'expected', u'expected', u'v')
(u'to', u'to', None)
(u'produce', u'produce', u'v')
(u'a', u'a', None)
(u'semantically', u'semantically', u'r')
(u'distant', u'distant', u'a')
(u'sentence', u'sentence', u'n')
(u'?', u'?', None)


In [4]:
# Loading the Wordnet domains.
from collections import defaultdict

from glob import glob
import os.path


domain_files = glob('../../../Resources/tools/lib/databases/xwnd-30g/*.ppv')
synset2domains = defaultdict(lambda: np.zeros(len(domain_files)))
for domain_index,domain_filename in enumerate(domain_files):
    domain = os.path.splitext(os.path.basename(domain_filename))[0]
    with open(domain_filename,'r') as fh:
        for line in list(fh):
            ssid, weight = line.split()
            weight = float(weight)
            #assert not np.isnan(weight)
            
            synset2domains[ssid][domain_index] = weight 


def get_domains(synset):
    return synset2domains[get_synset_id(synset)]

def get_synset_id(synset):
    return str(synset.offset()).zfill(8) + "-" + synset.pos()

In [29]:
np.power([1,2,3],4)

array([ 1, 16, 81])

In [151]:
def synset_counts(synsets):
    def count(synset):
        return sum([1.0*lemma.count() for lemma in synset.lemmas()])
    
    counts = np.asarray([count(ss) for ss in synsets])
    return counts

def synset_freqs(synsets):
    def count(synset):
        return sum([1.0*lemma.count() for lemma in synset.lemmas()])
    
    counts = np.asarray([count(ss) for ss in synsets])
    counts+=1.0
    counts/=counts.sum()
    return counts
    
def get_scaled_domains(synsets):
    counts= synset_freqs(synsets)
    counts = np.power(counts,2) #Squaring weighter bette than not, and better than cubing
    domains = [counts[ii]*get_domains(ss) for ii,ss in enumerate(synsets)]
    return domains


In [112]:
synset_misses = set()
def try_hard_to_get_synsets(word, pos):
    synsets = wn.synsets(word,pos)

#    if len(synsets)==0 and word.isdigit():
#        if 1900<int(word)<2020:
#            synsets = [wn.synset('modern_era.n.01'),wn.synset('turn_of_the_century.n.01')]
    
#    if len(synsets)==0:
#        synsets = wn.synsets(word.replace(".",""),pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.replace("-","_"),pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.replace("-",""),pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.replace("_",""),pos)
#    if len(synsets)==0 and word=='%':
#        synsets = wn.synsets('percent',pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.split("_")[-1],pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.split("-")[-1],pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.split("_")[0],pos)
#    if len(synsets)==0:
#        synsets = wn.synsets(word.split("-")[0],pos)
        
#    if len(synsets)==0 and not pos is None:
#        synsets = try_hard_to_get_synsets(word, None)
        
#    if len(synsets)==0:
#        synset_misses.add((word,pos))
    
        
    return synsets

def tag2wn_pos(tag):
    tag = 'a' if tag[0] =='J' else tag
    tag = 'n' if tag =='CD' else tag
    tag = '*' if tag =='VHP' else tag #Blank it
    tag = tag[0].lower()
    return tag if tag in ['a','v','r','n'] else None

In [152]:
def mfs_wsd(tagged_phrases):

    def get_best_synsets():
        for lemma,pos in tagged_phrases:
            best_synset = None
            wn_pos = tag2wn_pos(pos)
            if wn_pos:
                synsets = try_hard_to_get_synsets(lemma, wn_pos)
                if synsets:
                    scores=synset_counts(synsets)
                    best_synset = synsets[np.argmax(scores)]
            yield lemma, best_synset
                
            
    return list(get_best_synsets())

In [ ]:
def domain_wsd(tagged_phrases):
    sentence_domain = np.zeros(len(domain_files))

    for lemma,pos in tagged_phrases:
        wn_pos = tag2wn_pos(pos)
        if wn_pos: 
            domain_scores=get_scaled_domains(try_hard_to_get_synsets(lemma, wn_pos))
            for domain_score in domain_scores:
                sentence_domain+=domain_score
    
    def get_best_synsets():
        for lemma,pos in tagged_phrases:
            best_synset = None
            wn_pos = tag2wn_pos(pos)
            if wn_pos:
                best_score = -1*np.Inf
                synsets = try_hard_to_get_synsets(lemma, wn_pos)
                domain_scores=get_scaled_domains(synsets)
                for ii,ss in enumerate(synsets):
                    score = np.dot(domain_scores[ii],sentence_domain)
                    if score>best_score:
                        best_score=score
                        best_synset=ss
            #yield lemma, best_synset, None if best_synset is None else best_synset.definition()
            yield lemma, best_synset
            
    return list(get_best_synsets())

In [117]:
def wsd_on_plaintext(sent, wsd, max_phrase_len=3):
    words, tagged_words = tokenize_and_tag(sent)
    tagged_phrases = get_tagged_phrases(tagged_words,max_phrase_len)
    return [(lemma,ss,ss.definition() if ss else None) for lemma, ss in wsd(tagged_phrases)]
    

In [109]:
sent="Domestic sales at both GM and No. 2 Ford Motor Co. declined more than predicted as a late summer sales frenzy prompted a larger-than-expected industry backlash."
altered_sent="Domestic sales at both GM and No . 2 Ford Motor Co. declined more than predicted as a late summer sales frenzy prompted a larger-than-expected diligence backlash ."

In [110]:
domain_wsd_on_plaintext(sent)

[(u'Domestic',
  Synset('domestic.a.01'),
  u'of concern to or concerning the internal affairs of a nation'),
 (u'sales', Synset('sale.n.01'), u'a particular instance of selling'),
 (u'at', None, None),
 (u'both', None, None),
 (u'GM',
  Synset('gram.n.01'),
  u'a metric unit of weight equal to one thousandth of a kilogram'),
 (u'and', None, None),
 (u'No', Synset('no.n.01'), u'a negative'),
 (u'.', None, None),
 (u'2',
  Synset('two.n.01'),
  u'the cardinal number that is the sum of one and one or a numeral representing this number'),
 (u'Ford',
  Synset('ford.n.06'),
  u'United States manufacturer of automobiles who pioneered mass production (1863-1947)'),
 (u'Motor',
  Synset('motor.n.01'),
  u'machine that converts other forms of energy into mechanical energy and so imparts motion'),
 (u'Co.', None, None),
 (u'declined', Synset('refuse.v.01'), u'show unwillingness towards'),
 (u'more_than', None, None),
 (u'predicted',
  Synset('predict.v.01'),
  u'make a prediction about; tell in 

In [126]:
def get_sense_key(synset, lemma):
    index = 0
    try: #try and do better
        index = synset.lemma_names().index(lemma)
    except ValueError:
        pass #couldn't do better
    return synset.lemmas()[index].key()

def eval_wsd(wsd, text_id, tagged_phrases, indexed_instances):
    lemmas_and_synsets = wsd(tagged_phrases)
    for ii, id in indexed_instances.items():
        #print(id)
        lemma, synset = lemmas_and_synsets[ii]
        comment = "!! lemma="+''.join([i if ord(i) < 128 else ' ' for i in lemma])
        if synset: #If it was even defined in wordnet
            sensekey = get_sense_key(synset,lemma)
            yield text_id, id, sensekey, comment
        else:
            yield text_id, id, comment
        

In [127]:
import xml.etree.ElementTree as ET
def tagged_sentences(filename):
    xml = ET.ElementTree(file=filename)
    for text in xml.getroot():
        text_id = text.get('id')
        
        for sentence in text:
            tagged_phrases = []
            indexed_instances = dict()
            for ii,phrase in enumerate(sentence):
                lemma = phrase.get('lemma')
                lemma = phrase.text if lemma=="@card@" else lemma  #correct it to use the text if it was a number
                
                tagged_phrases.append((lemma,phrase.get('pos')))
                if 'id' in phrase.keys():
                    indexed_instances[ii] = phrase.get('id')
            yield text_id, tagged_phrases, indexed_instances    
    

def run_full_trial(wsd, corpus_filename, savename):
    answers = list(itertools.chain(*[eval_wsd(wsd, *tti) for tti in tagged_sentences(corpus_filename)]))
    with open("./SemEval13_task12_data/"+savename+".txt",'w') as fh:
        for answer_parts in answers:
            fh.write(" ".join(answer_parts)+'\n')
    return answers

In [131]:
corpus_filename = "./SemEval13_task12_data/test.en.xml"

In [153]:
run_full_trial(mfs_wsd, corpus_filename, "mfs_count")

[('d001', 'd001.s001.t001', '!! lemma=U.N.'),
 ('d001', 'd001.s001.t002', u'group%1:03:00::', '!! lemma=group'),
 ('d001', 'd001.s001.t003', u'plan%1:09:00::', '!! lemma=plan'),
 ('d001', 'd001.s001.t004', u'emission%1:04:00::', '!! lemma=emission'),
 ('d001', 'd001.s002.t011', u'year%1:28:01::', '!! lemma=year'),
 ('d001', 'd001.s002.t001', u'climate%1:26:01::', '!! lemma=climate'),
 ('d001', 'd001.s002.t002', u'conference%1:14:00::', '!! lemma=conference'),
 ('d001', 'd001.s002.t012', '!! lemma=industrialized_nation'),
 ('d001', 'd001.s002.t013', u'burden%1:06:01::', '!! lemma=burden'),
 ('d001', 'd001.s002.t003', u'posturing%1:07:00::', '!! lemma=posturing'),
 ('d001',
  'd001.s002.t004',
  u'recrimination%1:10:00::',
  '!! lemma=recrimination'),
 ('d001', 'd001.s002.t014', u'term%1:10:00::', '!! lemma=term'),
 ('d001', 'd001.s002.t005', u'focus%1:09:00::', '!! lemma=focus'),
 ('d001', 'd001.s002.t006', u'friday%1:28:00::', '!! lemma=Friday'),
 ('d001', 'd001.s002.t007', u'release%1

In [80]:
s

Synset('turn_of_the_century.n.01')

In [150]:
!git add WSD.ipynb
!git commit -m="WSD using most frequent sense"
!git push

[master 1ad80c0] =WSD using most frequent sense
 1 file changed, 1359 insertions(+), 745 deletions(-)
To git@github.com:oxinabox/phd.git
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to 'git@github.com:oxinabox/phd.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.
